This is the final model that we will be using to predict the classes of the test images

The final model is a stacking model with the following pre-trained CNN architectures as base classifiers:

* resnet34
* vgg19
* densenet161
* resnext101_32x8d
* vgg16

The meta classifier is a kernel SVM with a radial basis function (rbf)

The pre-trained CNNs were trained in separate notebooks using fastai2 which is a library for deep learning.

The probabilities predicted by each neural network was saved as a torch tensor and is used to train the meta classifier.

*Importing The Libraries*

In [8]:
import torch
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy
import numpy as np
import pandas as pd

*Load the tensors into our program*

###### please change the directory to ensure there aren't any errors

In [2]:
resnet34_val_tensor = torch.load('/content/proba_valid_resnet34')
resnet34_test_tensor = torch.load('/content/proba_test_resnet34')
vgg19_val_tensor = torch.load('/content/proba_valid_vgg19')
vgg19_test_tensor = torch.load('/content/proba_test_vgg19')
densenet161_val_tensor = torch.load('/content/proba_valid_densenet161')
densenet161_test_tensor = torch.load('/content/proba_test_densenet161')
resnext101_32x8d_val_tensor = torch.load('/content/proba_valid_resnext101_32x8d')
resnext101_32x8d_test_tensor = torch.load('/content/proba_test_resnext101_32x8d')
vgg16_val_tensor = torch.load('/content/proba_valid_vgg16')
vgg16_test_tensor = torch.load('/content/proba_test_vgg16')

*Convert the torch tensors into numpy arrays*

In [3]:
resnet34_val_array = resnet34_val_tensor.numpy()
resnet34_test_array = resnet34_test_tensor.numpy()

vgg19_val_array = vgg19_val_tensor.numpy()
vgg19_test_array = vgg19_test_tensor.numpy()

densenet161_val_array = densenet161_val_tensor.numpy()
densenet161_test_array = densenet161_test_tensor.numpy()

resnext101_32x8d_val_array = resnext101_32x8d_val_tensor.numpy()
resnext101_32x8d_test_array = resnext101_32x8d_test_tensor.numpy()

vgg16_val_array = vgg16_val_tensor.numpy()
vgg16_test_array = vgg16_test_tensor.numpy()

*Generate the training and test input array*

In [4]:
train = numpy.hstack((resnet34_val_array, vgg19_val_array,
                      densenet161_val_array, resnext101_32x8d_val_array,
                      vgg16_val_array))

test = numpy.hstack((resnet34_test_array, vgg19_test_array,
                     densenet161_test_array, resnext101_32x8d_test_array,
                     vgg16_test_array))

*Printing the shapes of our arrays to verify that we haven't made any mistakes*

In [5]:
print(train.shape)

(21079, 210)


In [6]:
print(test.shape)

(12186, 210)


*Load the classes of our training data in order to train our SVM*

In [9]:
dataset = pd.read_csv("meta_labels.csv")
labels = dataset.iloc[:, -1].values

In [10]:
print(labels.shape)

(21079,)


*Splitting the data into train and test*

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size = 0.1)

*Training our SVM on 90% of the train data*

In [12]:
from sklearn.svm import SVC
classifier = SVC(kernel='rbf')
classifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [13]:
pred = classifier.predict(X_test)
print("Accuracy on validation data: ",accuracy_score(pred, y_test))

Accuracy on validation data:  0.8221062618595826


The accuracy of the model is good.
We decided to train an SVM on the entire training data as there is not much training data and there wasn't any evident signs of overfitting judging by the validation(X_test, y_test) accuracy

In [14]:
meta_classifier = SVC(kernel='rbf')
meta_classifier.fit(train, labels)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [15]:
test_pred = classifier.predict(test)
print(test_pred)

[20 27  4 ... 13 15 23]


*The final .csv file for submission*

In [16]:
test_csv = pd.read_csv("test.csv")

submission  = test_csv[["filename"]]
submission["category"] = test_pred
submission["category"] = submission.category.apply(lambda c: str(c).zfill(2))

submission.to_csv("svm_submissionlast1.csv", index=False)